In [8]:
import pybaldes as baldes
import json
import random
import uuid
from datetime import datetime, timedelta
import numpy as np
from collections import defaultdict

In [ ]:


# Convert a datetime to minutes since epoch
def datetime_to_minutes(dt):
    return int(dt.timestamp() // 60)

# Generate random minutes within a given range
def generate_random_minutes(start_minutes, end_minutes):
    return int(random.uniform(start_minutes, end_minutes))

# Generate view periods for resources
def generate_view_periods(start_minutes, end_minutes, num_periods=3):
    periods = []
    period_duration = (end_minutes - start_minutes) // num_periods
    for i in range(num_periods):
        rise = start_minutes + period_duration * i
        set_time = rise + int(random.uniform(60, 240))  # Duration between 1 to 4 hours in minutes
        periods.append({
            "RISE": rise,
            "SET": set_time,
            "TRX ON": rise,
            "TRX OFF": set_time
        })
    return periods

# Generate track data for a given week and year
def generate_track_data(week, year, num_tracks):
    start_week = datetime.strptime(f'{year} {week} 1', '%Y %W %w')
    start_week_minutes = 0
    data = []
    for _ in range(num_tracks):
        duration = int(random.uniform(60, 2*60))  # Duration between 1 to 3.5 hours in minutes
        setup_time = 60
        teardown_time = 15
        track_id = str(uuid.uuid4())
        time_window_start = start_week_minutes + generate_random_minutes(0, 6.5 * 24 * 60 * 1)  # Random start within the week
        time_window_end = time_window_start + generate_random_minutes(1 * 60, 1.5 * 60)  # Random end 1 to 2 hours later
        
        if random.random() < 0.0:
            resources = random.sample(["DSS-24", "DSS-26", "DSS-34", "DSS-36", "DSS-54"], k=2)
            # concatenate name with _
            resources = ["_".join(resources)]
        else:
            resources = random.sample(["DSS-24", "DSS-26", "DSS-34", "DSS-36", "DSS-54"], k=random.randint(1, 5))
        resource_vp_dict = {res: generate_view_periods(time_window_start, time_window_end) for res in resources}
        
        track = {
            "subject": 521,
            "user": "521_0",
            "week": week,
            "year": year,
            "duration": duration,
            "duration_min": duration,
            "resources": [[res] for res in resources],
            "track_id": track_id,
            "setup_time": setup_time,
            "teardown_time": teardown_time,
            "time_window_start": time_window_start,
            "time_window_end": time_window_end,
            "resource_vp_dict": resource_vp_dict
        }
        data.append(track)
    return data


# Generate DSN data for a range of weeks and a given year
def generate_dsn_data(weeks, year, num_tracks_per_week):
    dsn_data = {}
    for week in weeks:
        week_key = f"W{week}_{year}"
        dsn_data[week_key] = generate_track_data(week, year, num_tracks_per_week)
    return dsn_data

weeks = [1]
year = 2018
num_tracks_per_week = 20
dsn_data = generate_dsn_data(weeks, year, num_tracks_per_week)

with open("build/toy_problem.json", "w") as file:
    json.dump(dsn_data, file, indent=2)

    print("DSN test data generated successfully.")


DSN test data generated successfully.


In [2]:
import json

class Instance:
    def __init__(self):
        self.tracks = []          # List of track IDs
        self.resources = set()    # Set of antenna resources
        self.track_nodes = {}     # {track_id: node_data}
        self.sync_groups = []     # Groups requiring simultaneous scheduling

    def load_data(self, file_path):
        """Load and parse DSN scheduling data from JSON file"""
        self.__init__()
        with open(file_path) as f:
            raw_data = json.load(f)
            
        for week_data in raw_data.values():
            for track in week_data:
                self._process_track(track)

    def _process_track(self, track):
        """Create node with embedded resource-timewindow data"""
        track_id = track['track_id']
        resource_windows = {}
        current_sync_groups = []

        # Process each resource group
        for res_group in track['resources']:
            res = res_group[0]  # Get resource string
            
            if '_' in res:  # Handle sync groups
                antennas = res.split('_')
                current_sync_groups.append(antennas)
                
                # Add windows to each antenna in group
                combined_periods = track['resource_vp_dict'].get(res, [])
                for antenna in antennas:
                    self.resources.add(antenna)
                    periods = [(p['TRX ON'], p['TRX OFF']) for p in combined_periods]
                    resource_windows.setdefault(antenna, []).extend(periods)
            else:  # Single antenna
                self.resources.add(res)
                periods = [(p['TRX ON'], p['TRX OFF']) 
                          for p in track['resource_vp_dict'].get(res, [])]
                resource_windows[res] = periods

        # Update global sync groups
        for group in current_sync_groups:
            if group not in self.sync_groups:
                self.sync_groups.append(group)

        # Store track data
        self.track_nodes[track_id] = {
            'duration': track['duration_min'],
            'setup': track['setup_time'],
            'teardown': track['teardown_time'],
            'mission_window': (track['time_window_start'], track['time_window_end']),
            'resource_windows': resource_windows,
            'sync_groups': current_sync_groups
        }
        self.tracks.append(track_id)

# Create and load instance
dsn_instance = Instance()
# dsn_instance.load_data("build/toy_problem.json")
dsn_instance.load_data("build/dsn_schedule.json")

In [3]:
import itertools

class Graph:
    def __init__(self, instance):
        self.instance = instance
        self.nodes = {}       # Track nodes + virtual nodes
        self.edges = {}       # Edges with travel times
        self.resources = set()

        self._build_nodes()
        self._build_edges()

    def _build_nodes(self):
        """Add virtual start/end nodes and store resource windows in track nodes"""
        # Regular track nodes with resource-specific time windows
        for track_id in self.instance.tracks:
            track_data = self.instance.track_nodes[track_id]
            self.nodes[track_id] = {
                'type': 'track',
                'duration': track_data['duration'],
                'setup': track_data['setup'],
                'teardown': track_data['teardown'],
                'mission_window': track_data['mission_window'],
                'resource_windows': track_data['resource_windows'],
                'resources': list(track_data['resource_windows'].keys())
            }
            self.resources.update(track_data['resource_windows'].keys())

        # Virtual nodes for each antenna
        for antenna in self.resources:
            vs_node = f"vs_{antenna}"
            ve_node = f"ve_{antenna}"
            
            self.nodes[vs_node] = {
                'type': 'virtual_start',
                'setup': 0,
                'teardown': 0,
                'duration': 0,
                'time_window': (0, float('inf')),
                'resources': [antenna]
            }
            
            self.nodes[ve_node] = {
                'type': 'virtual_end',
                'setup': 0,
                'teardown': 0,
                'duration': 0,
                'time_window': (0, float('inf')),
                'resources': [antenna]
            }

    def _get_travel_time(self, u, v):
        """Calculate transition time between nodes"""
        if self.nodes[u]['type'] == 'virtual_start':
            return self.nodes[v]['setup']
        elif self.nodes[v]['type'] == 'virtual_end':
            return self.nodes[u]['teardown']
        return self.nodes[u]['teardown'] + self.nodes[v]['setup']

    def _can_follow(self, u, v, antenna):
        """Check if v can follow u on antenna, considering resource windows"""
        if 'virtual' in u or 'virtual' in v:
            return False

        # Both tracks must support this antenna
        if antenna not in self.nodes[u]['resource_windows'] or \
           antenna not in self.nodes[v]['resource_windows']:
            return False

        # Check time window compatibility
        for u_window in self.nodes[u]['resource_windows'][antenna]:
            u_trx_on, u_trx_off = u_window
            u_total_end = u_trx_on + self.nodes[u]['duration']
            min_v_start = u_total_end + self._get_travel_time(u, v)

            for v_window in self.nodes[v]['resource_windows'][antenna]:
                v_trx_on, v_trx_off = v_window
                latest_v_start = v_trx_off - self.nodes[v]['duration']

                if min_v_start <= latest_v_start:
                    return True
        return False

    def _build_edges(self):
        """Build edges considering resource-specific windows"""
        for antenna in self.resources:
            vs_node = f"vs_{antenna}"
            ve_node = f"ve_{antenna}"

            # Add idle edge
            self._add_edge(vs_node, ve_node, antenna, 0)

            # Get tracks supporting this antenna
            tracks_on_a = [
                t for t in self.instance.tracks 
                if antenna in self.nodes[t]['resource_windows']
            ]

            # Track-to-track edges
            for u, v in itertools.permutations(tracks_on_a, 2):
                if self._can_follow(u, v, antenna):
                    self._add_edge(u, v, antenna, self._get_travel_time(u, v))

            # Connect virtual nodes
            for track in tracks_on_a:
                if self._can_start(track, antenna):
                    self._add_edge(vs_node, track, antenna, self.nodes[track]['setup'])
                self._add_edge(track, ve_node, antenna, self.nodes[track]['teardown'])

    def _can_start(self, track, antenna):
        """Check if track can be first on antenna"""
        setup_time = self.nodes[track]['setup']
        return any(
            setup_time <= window[0] 
            for window in self.nodes[track]['resource_windows'][antenna]
        )

    def _add_edge(self, u, v, antenna, travel_time):
        """Store edge with transition metadata"""
        key = (u, v)
        if key not in self.edges:
            self.edges[key] = {}
        self.edges[key][antenna] = travel_time

    def get_feasible_successors(self, u, antenna):
        """Get all valid next nodes (including ve_a)"""
        successors = []
        for (src, dest), antennas in self.edges.items():
            if src == u and antenna in antennas:
                successors.append(dest)
        return successors

    def get_feasible_predecessors(self, v, antenna):
        """Get all valid previous nodes (including vs_a)"""
        predecessors = []
        for (src, dest), antennas in self.edges.items():
            if dest == v and antenna in antennas:
                predecessors.append(src)
        return predecessors
    
    def build_distance_matrix(self):
        """Build a distance matrix for the graph"""
        num_nodes = len(self.nodes)
        distance_matrix = []
        for i in range(num_nodes):
            row = []
            for j in range(num_nodes):
                if i == j:
                    row.append(0)
                else:
                    u = list(self.nodes.keys())[i]
                    v = list(self.nodes.keys())[j]
                    travel_time = self._get_travel_time(u, v)
                    row.append(travel_time)
            distance_matrix.append(row)
        return distance_matrix
    
    def print_graph(self):
        """Print graph structure with nodes and edges, including resource windows"""
        print("="*40)
        print("Graph Nodes:")
        for node_id, data in self.nodes.items():
            node_type = data['type']
            if node_type == 'track':
                print(f"\nTrack {node_id}:")
                print(f"  Duration: {data['duration']} min")
                print(f"  Setup: {data['setup']} min | Teardown: {data['teardown']} min")
                print(f"  Mission Window: {data['mission_window'][0]}–{data['mission_window'][1]}")
                print("  Resource Windows:")
                for resource, windows in data['resource_windows'].items():
                    window_str = ", ".join([f"{start}–{end}" for start, end in windows])
                    print(f"    {resource}: {window_str}")
                print(f"  Sync Groups: {self.instance.track_nodes[node_id].get('sync_groups', [])}")
            else:
                print(f"\nVirtual {node_type.split('_')[-1]} Node {node_id}:")
                print(f"  Resources: {data['resources']}")

        print("\n" + "="*40)
        print("Graph Edges:")
        for (src, dest), antennas in self.edges.items():
            for antenna, travel_time in antennas.items():
                arrow = f"[{antenna}] {travel_time}min"
                print(f"{src: <20} —— {arrow: <25} → {dest}")

instance = Instance()
instance.load_data("build/toy_problem.json")
graph = Graph(instance)
graph.print_graph()


Graph Nodes:

Track ef05fbcd-5daf-4adf-9419-fedd7b225576:
  Duration: 102 min
  Setup: 60 min | Teardown: 15 min
  Mission Window: 3181–3258
  Resource Windows:
    DSS-36: 3181–3243, 3206–3385, 3231–3340
    DSS-26: 3181–3270, 3206–3314, 3231–3378
    DSS-24: 3181–3334, 3206–3297, 3231–3348
    DSS-54: 3181–3324, 3206–3391, 3231–3437
  Sync Groups: []

Track 6d67e9b2-2364-42cc-b3c5-d6dad65687b0:
  Duration: 102 min
  Setup: 60 min | Teardown: 15 min
  Mission Window: 6273–6355
  Resource Windows:
    DSS-24: 6273–6433, 6300–6538, 6327–6522
    DSS-26: 6273–6476, 6300–6485, 6327–6464
    DSS-34: 6273–6360, 6300–6446, 6327–6409
    DSS-54: 6273–6428, 6300–6498, 6327–6416
  Sync Groups: []

Track c3ced005-6acf-4d99-9a2e-3b9f5927a55d:
  Duration: 72 min
  Setup: 60 min | Teardown: 15 min
  Mission Window: 9330–9404
  Resource Windows:
    DSS-24: 9330–9447, 9354–9530, 9378–9568
    DSS-54: 9330–9408, 9354–9463, 9378–9490
    DSS-34: 9330–9400, 9354–9468, 9378–9511
    DSS-26: 9330–9398, 9

In [4]:
from gurobipy import Model, GRB, quicksum

class OptimizationProblem:
    def __init__(self, graph):
        self.graph = graph
        self.model = Model("DSN_Scheduling")
        self.M = 1e5
        
        # Decision variables from Table 2
        self.y = {}          # y_i (track scheduled)
        self.x = {}          # x^a_{u,v} (antenna flow)
        self.z = {}          # z^a_{u,w} (window selection)
        self.tau = {}        # τ_{u,a} (start time)
        
        self._create_variables()
        self._build_constraints()

    def _create_variables(self):
        """Create all decision variables"""
        # Activity scheduling variables
        for track in self.graph.instance.tracks:
            self.y[track] = self.model.addVar(vtype=GRB.BINARY, name=f"y_{track}")

        # Antenna flow variables
        for (u, v), antennas in self.graph.edges.items():
            for a in antennas:
                self.x[(u, v, a)] = self.model.addVar(
                    vtype=GRB.BINARY, name=f"x_{u}_{v}_{a}"
                )

        # Time window selection variables
        for track in self.graph.instance.tracks:
            for a in self.graph.nodes[track]['resources']:
                windows = self.graph.nodes[track]['resource_windows'][a]
                for w_idx in range(len(windows)):
                    self.z[(track, a, w_idx)] = self.model.addVar(
                        vtype=GRB.BINARY, name=f"z_{track}_{a}_{w_idx}"
                    )

        # Start time variables
        for track in self.graph.instance.tracks:
            for a in self.graph.nodes[track]['resources']:
                self.tau[(track, a)] = self.model.addVar(
                    vtype=GRB.CONTINUOUS, name=f"tau_{track}_{a}"
                )

    def _build_constraints(self):
        """Build all constraints"""
        self._build_flow_constraints()
        print("Flow constraints built.")
        self._build_window_selection_constraints()
        print("Window selection constraints built.")
        self._build_timing_constraints()
        print("Timing constraints built.")
        # self._build_sync_constraints()
        # print("Sync constraints built.")

    def _build_sync_constraints(self):
        """Constraints for synchronized antenna groups"""
        for track_id in self.graph.instance.tracks:
            track_data = self.graph.instance.track_nodes[track_id]
            
            # Skip non-synchronized tracks
            if not track_data['sync_groups']:
                continue
            else:
                print(f"Sync groups for track {track_id}: {track_data['sync_groups']}")
                
            # For each sync group (e.g., ["DSS-26", "DSS-24"])
            for group in track_data['sync_groups']:
                print(f"Processing sync group {group} for track {track_id}")
                a1, a2 = group

                # 1. Both antennas must be assigned to this track
                self.model.addConstr(
                    quicksum(self.x[(u, track_id, a1)] 
                    for u in self.graph.get_feasible_predecessors(track_id, a1))
                    == self.y[track_id],
                    name=f"sync_assign_{track_id}_{a1}"
)
                
                self.model.addConstr(
                    quicksum(self.x[(u, track_id, a2)] 
                    for u in self.graph.get_feasible_predecessors(track_id, a2))
                    == self.y[track_id],
                    name=f"sync_assign_{track_id}_{a2}"
                )
                
                self.model.addConstr(
                    self.tau[(track_id, a1)] == self.tau[(track_id, a2)],
                    name=f"sync_time_{track_id}"
                )

                # 3. Same window selection (using window indices)
                windows_a1 = track_data['resource_windows'][a1]
                windows_a2 = track_data['resource_windows'][a2]
                for w in range(len(windows_a1)):
                    self.model.addConstr(
                        self.z[(track_id, a1, w)] == self.z[(track_id, a2, w)],
                        name=f"sync_window_{track_id}_{w}"
                    )
                    


    def _build_flow_constraints(self):
        """Antenna path and flow conservation constraints"""
        # Original flow constraints from previous implementation
        for antenna in self.graph.resources:
            vs = f"vs_{antenna}"
            ve = f"ve_{antenna}"
            
            # Start/end node constraints
            self.model.addConstr(
                quicksum(self.x[(vs, v, antenna)] for v in self.graph.get_feasible_successors(vs, antenna)) == 1,
                name=f"start_{antenna}"
            )
            self.model.addConstr(
                quicksum(self.x[(u, ve, antenna)] for u in self.graph.get_feasible_predecessors(ve, antenna)) == 1,
                name=f"end_{antenna}"
            )
            # Flow conservation constraints

        # Link flow variables to activity scheduling
        for u in self.graph.nodes:
            if self.graph.nodes[u]['type'] == 'track':
                for a in self.graph.nodes[u]['resources']:
                        self.model.addConstr(
                            quicksum(self.x[(v, u, a)] for v in self.graph.get_feasible_predecessors(u, a))
                            == quicksum(self.x[(u, v, a)] for v in self.graph.get_feasible_successors(u,a)),
                            name=f"flow_conservation_{u}_{a}"
                        )
                        self.model.addConstr(
                            quicksum(self.x[(u, v, a)] for v in self.graph.get_feasible_successors(u,a)) <= self.y[u],
                            name=f"flow_conservation_{u}_{a}_less_than_1"
                        )
        
        for u in self.graph.nodes:
            if self.graph.nodes[u]['type'] == 'track':
                track_data = self.graph.instance.track_nodes[u]
                if not track_data['sync_groups']:
                    self.model.addConstr(
                        quicksum(self.x[(u, v, a)] for a in self.graph.nodes[u]['resources'] for v in self.graph.get_feasible_successors(u,a)) 
                        <= self.y[u],
                        name=f"activity_scheduling_{u}"
                    )
                else:
                    self.model.addConstr(
                        quicksum(self.x[(u, v, a)] for a in self.graph.nodes[u]['resources'] for v in self.graph.get_feasible_successors(u,a)) 
                        == 2 * self.y[u],
                        name=f"activity_scheduling_{u}"
                    )

    def _build_window_selection_constraints(self):
        """Time window selection constraints"""
        for u in self.graph.nodes:
            if self.graph.nodes[u]['type'] == 'track':
                for a in self.graph.nodes[u]['resources']:
                    windows = self.graph.nodes[u]['resource_windows'][a]
                    self.model.addConstr(
                        quicksum(self.z[(u, a, w)] for w in range(len(windows))) 
                        == quicksum(self.x[(u, v, a)] for v in self.graph.get_feasible_successors(u,a)),
                        name=f"window_select_{u}_{a}"
                    )
                    self.model.addConstr(
                        quicksum(self.z[(u, a, w)] for w in range(len(windows))) == self.y[u],
                        name=f"window_select_{u}_{a}_equals_y"
                    )

    def _build_timing_constraints(self):
        """Timing constraints with window enforcement"""
        for u in self.graph.nodes:
            if self.graph.nodes[u]['type'] == 'track':
                for a in self.graph.nodes[u]['resources']:
                    for v in self.graph.get_feasible_successors(u, a):
                        if self.graph.nodes[v]['type'] == 'track':
                            travel_time = self.graph.edges[(u, v)][a]
                            for w_idx, (trx_on, trx_off) in enumerate(self.graph.nodes[u]['resource_windows'][a]):
                                self.model.addConstr(
                                    self.tau[(v, a)] >= self.tau[(u, a)] + self.graph.nodes[u]['duration'] + travel_time + self.M * (self.x[(u, v, a)] - 1),
                                    name=f"timing_{u}_{v}_{a}_{w_idx}"
                                )
        for u in self.graph.nodes:
            if self.graph.nodes[u]['type'] == 'track':
                for a in self.graph.nodes[u]['resources']:
                    for w_idx, (trx_on, trx_off) in enumerate(self.graph.nodes[u]['resource_windows'][a]):
                        self.model.addConstr(
                            self.tau[(u, a)] >= trx_on + self.M * (self.z[(u, a, w_idx)] - 1),
                            name=f"window_start_{u}_{a}_{w_idx}"
                        )
                        self.model.addConstr(
                            self.tau[(u, a)] + self.graph.nodes[u]['duration'] <= trx_off + self.M * (1 - self.z[(u, a, w_idx)]),
                            name=f"window_end_{u}_{a}_{w_idx}"
                        )

    def set_objective(self):
        """Example objective: Maximize total scheduled duration"""
        total_duration = quicksum(
            self.y[track] * self.graph.nodes[track]['duration']
            for track in self.graph.instance.tracks
        )
        self.model.setObjective(total_duration, GRB.MAXIMIZE)

    def solve(self):
        """Solve and return status"""
        self.model.optimize()
        return self.model.status
    
    def print_solution(self):
        """Print the solution with sync track handling"""
        if self.model.status != GRB.OPTIMAL:
            print("No optimal solution found.")
            return

        def time_str(mins):
            return f"{int(mins//60):02d}:{int(mins%60):02d}"

        print("\n=== Optimal Schedule ===")
        scheduled_sync_tracks = set()

        # Print synchronized tracks first
        print("\nSYNCHRONIZED TRACKS:")
        for track_id in self.graph.instance.tracks:
            # Get data from GRAPH nodes, not instance
            track_node = self.graph.nodes[track_id]
            if not track_node.get('sync_groups') or self.y[track_id].X < 0.5:
                continue

            scheduled_sync_tracks.add(track_id)
            group = track_node['sync_groups'][0]
            a1, a2 = group[0], group[1]

            # Get common timing information
            start = self.tau[(track_id, a1)].X
            end = start + track_node['duration']
            w_idx = next(i for i in range(len(track_node['resource_windows'][a1])) 
                        if self.z[(track_id, a1, i)].X > 0.5)
            window = track_node['resource_windows'][a1][w_idx]

            print(f"\n🚀 Track {track_id} (SYNC)")
            print(f"  Antennas: {', '.join(group)}")
            print(f"  Start: {time_str(start)}")
            print(f"  End: {time_str(end)}")
            print(f"  Duration: {track_node['duration']}min")
            print(f"  Selected Window: {time_str(window[0])}-{time_str(window[1])}")

        # Print regular tracks
        print("\nREGULAR TRACKS:")
        for track_id in self.graph.instance.tracks:
            if self.y[track_id].X < 0.5 or track_id in scheduled_sync_tracks:
                continue

            # Get data from GRAPH nodes
            track_node = self.graph.nodes[track_id]
            print(f"\n📡 Track {track_id}")
            print(f"  Duration: {track_node['duration']}min")
            
            # Access resources from graph node
            for a in track_node['resources']:
                if any(self.x[(u, track_id, a)].X > 0.5 for u in self.graph.get_feasible_predecessors(track_id, a)):
                    w_idx = next(i for i in range(len(track_node['resource_windows'][a])) 
                            if self.z[(track_id, a, i)].X > 0.5)
                    start = self.tau[(track_id, a)].X
                    end = start + track_node['duration']
                    window = track_node['resource_windows'][a][w_idx]
                    
                    print(f"  On {a}:")
                    print(f"    TRX: {time_str(start)}-{time_str(end)}")
                    print(f"    Window: {time_str(window[0])}-{time_str(window[1])}")

    def print_antenna_paths(self):
        """Print antenna paths with sync track awareness"""
        if self.model.status != GRB.OPTIMAL:
            print("No solution found")
            return

        def time_str(mins):
            return f"{int(mins//60):02d}:{int(mins%60):02d}"

        print("\nANTENNA PATHS:")
        for antenna in self.graph.resources:
            path = []
            current_node = f"vs_{antenna}"
            
            print(f"\n📻 {antenna} Timeline:")
            print(f"╠═ {current_node} [Start]")

            while True:
                # Find next node
                next_node = None
                for (u, v, a) in self.x:
                    if u == current_node and a == antenna and self.x[(u, v, a)].X > 0.5:
                        next_node = v
                        break

                if not next_node or next_node.startswith("ve_"):
                    break

                # Process track node
                track_id = next_node
                track_data = self.graph.instance.track_nodes[track_id]
                duration = track_data['duration']
                start = self.tau[(track_id, antenna)].X
                end = start + duration
                w_idx = next(i for i in range(len(track_data['resource_windows'][antenna])) 
                        if self.z[(track_id, antenna, i)].X > 0.5)
                window = track_data['resource_windows'][antenna][w_idx]

                # Check if sync track
                sync_note = ""
                if track_data['sync_groups']:
                    sync_partners = [a for a in track_data['sync_groups'][0] if a != antenna]
                    sync_note = f" (SYNC with {', '.join(sync_partners)})"

                print(f"╠═ Track {track_id}{sync_note}")
                print(f"║  ├─ Window: {time_str(window[0])}-{time_str(window[1])}")
                print(f"║  ├─ TRX: {time_str(start)}-{time_str(end)}")
                print(f"║  └─ Duration: {duration}min")
                
                current_node = track_id

            print(f"╚═ ve_{antenna} [End]")

optimizer = OptimizationProblem(graph)
optimizer.set_objective()  
status = optimizer.solve()

if status == GRB.OPTIMAL:
    optimizer.print_solution()
    optimizer.print_antenna_paths()

Set parameter Username
Set parameter LicenseID to value 2629297
Academic license - for non-commercial use only - expires 2026-02-27
Flow constraints built.
Window selection constraints built.
Timing constraints built.
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i7-1360P, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1411 rows, 632 columns and 5393 nonzeros
Model fingerprint: 0x4ca99ad7
Variable types: 55 continuous, 577 integer (577 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [6e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Found heuristic solution: objective -0.0000000
Presolve removed 1411 rows and 632 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.01 work units)
Threa

In [5]:
from collections import defaultdict
import numpy as np
import gurobipy as gp
import pybaldes as baldes
import random

class ColumnGenerationScheduler:
    def __init__(self, graph):
        self.graph = graph
        self.antenna_jobs = self._distribute_jobs_to_antennas()
        self.antennas = list(self.antenna_jobs.keys())
        self.all_jobs = self._get_all_jobs()
        
        # Data structures
        self.antenna_nodes = defaultdict(list)
        self.colunas = defaultdict(list)
        self.buckets = {}
        self.nos = {}
        
        # Build initial solution
        self._create_vrp_nodes()
        self._run_greedy_heuristic()
        self._create_initial_columns()
        self._setup_bucket_graphs()

    def _distribute_jobs_to_antennas(self):
        """Map graph nodes to antenna-specific jobs"""
        antenna_jobs = defaultdict(list)
        for track_id in self.graph.nodes:
            node_data = self.graph.nodes[track_id]
            if node_data['type'] != 'track':
                continue
            
            for antenna in node_data['resources']:
                antenna_jobs[antenna].append({
                    'track_id': track_id,
                    'duration': node_data['duration'],
                    'resource_windows': node_data['resource_windows'][antenna],
                    'setup': node_data['setup'],
                    'teardown': node_data['teardown']
                })
        return antenna_jobs

    def _get_all_jobs(self):
        return {job['track_id'] for antenna in self.antennas for job in self.antenna_jobs[antenna]}

    def _create_vrp_nodes(self):
        """Create VRP nodes using graph data"""
        for antenna in self.antennas:
            self.antenna_nodes[antenna] = []
            for job in self.antenna_jobs[antenna]:
                node = baldes.VRPNode()
                node.duration = job['duration']
                node.identifier = job['track_id']
                node.mtw_lb = [tw[0] for tw in job['resource_windows']]
                node.mtw_ub = [tw[1] for tw in job['resource_windows']]
                node.set_location(random.randint(0, 100), random.randint(0, 100))
                node.cost = 0  
                node.demand = 0  
                node.consumption = [0] 
                node.id = len(self.antenna_nodes[antenna])
                self.antenna_nodes[antenna].append(node)
                print(f"Node {node.identifier} created for antenna {antenna} with windows {node.mtw_lb} to {node.mtw_ub}")
        

    def _run_greedy_heuristic(self):
        """Generate initial solution using graph-compatible greedy heuristic"""
        self.already_selected = []
        self.initial_solution = {}
        for antenna, nodes in self.antenna_nodes.items():
            sorted_nodes = sorted(nodes, key=lambda n: min(n.mtw_lb))
            selected = []
            last_end = 0
            
            for node in sorted_nodes:
                if node.identifier in self.already_selected:
                    continue
                for i in range(len(node.mtw_lb)):
                    if node.mtw_lb[i] >= last_end:
                        selected.append({
                            'identifier': node.identifier,
                            'start': node.mtw_lb[i],
                            'end': node.mtw_ub[i]
                        })
                        self.already_selected.append(node.identifier)
                        last_end = node.mtw_ub[i]
                        break
            self.initial_solution[antenna] = selected
            print(f"Greedy solution for antenna {antenna}: {selected}")
        print("Greedy heuristic completed.")

    def _create_initial_columns(self):
        """Initialize columns from greedy solution"""
        for antenna in self.antennas:
            self.colunas[antenna] = [
                [job['identifier'] for job in self.initial_solution[antenna]]]
                
    def _setup_bucket_graphs(self):
        """Build bucket graphs using graph edges"""
        for antenna in self.antennas:
            vs_node = f"vs_{antenna}"
            ve_node = f"ve_{antenna}"
            track_nodes = self.antenna_nodes[antenna]
            ub = max(
                max(node.mtw_ub) for node in track_nodes
            ) + 200
            print(f"Max time for antenna {antenna}: {ub}")
            # Create virtual nodes from graph
            start = baldes.VRPNode()
            start.duration = 0
            start.identifier = 'start'
            start.cost = 0
            start.demand = 0
            start.consumption = [0]
            start.set_location(0, 0)
            start.mtw_lb = [0]
            start.mtw_ub = [ub]
            
            end = baldes.VRPNode()
            end.duration = 0
            end.identifier = 'end'
            end.cost = 0
            end.demand = 0
            end.consumption = [0]
            end.set_location(0, 0)
            end.mtw_lb = [0]
            end.mtw_ub = [ub]

            full_nodes = [start] + track_nodes + [end]
            
            # Build distance matrix from graph edges
            size = len(full_nodes)
            distance_matrix = self.graph.build_distance_matrix()

            # Configure bucket graph
            options = baldes.BucketOptions()
            options.depot = 0
            options.end_depot = len(full_nodes)-1
            options.size = len(full_nodes)
            options.resource_type = [3]
            options.bucket_fixing = False
            
            max_time = max(
                max(node.mtw_ub) for node in track_nodes
             for track in self.graph.instance.tracks) + 1000
            
            for node in full_nodes:
                node.lb = [min(node.mtw_lb)]
                node.ub = [max(node.mtw_ub)]

            counter = 0
            for node in full_nodes:
                node.id = counter
                counter += 1
            
            bg = baldes.BucketGraph(full_nodes, int(max_time), 1)
            bg.setOptions(options)
            bg.set_distance_matrix(distance_matrix)
            bg.setup()
            
            self.buckets[antenna] = bg
            self.nos[antenna] = full_nodes

    def _create_master_problem(self):
        """Build master problem with graph-compatible constraints"""
        master = gp.Model()
        master.Params.OutputFlag = 0
        
        # Create variables
        x = {}
        for antenna in self.antennas:
            for idx, col in enumerate(self.colunas[antenna]):
                x[(antenna, idx)] = master.addVar(
                    vtype=gp.GRB.CONTINUOUS,
                    name=f"x_{antenna}_{idx}",
                    obj=len(col)
                )

        # Job coverage constraints
        for job in self.all_jobs:
            lhs = gp.quicksum(
                x[(antenna, idx)]
                for antenna in self.antennas
                for idx, col in enumerate(self.colunas[antenna])
                if job in col
            )
            master.addConstr(lhs <= 1, f"job_{job}")

        # Antenna constraints
        for antenna in self.antennas:
            master.addConstr(
                gp.quicksum(x[(antenna, idx)] 
                for idx in range(len(self.colunas[antenna]))) <= 1,
                f"antenna_{antenna}"
            )

        master.modelSense = gp.GRB.MAXIMIZE
        master.write("master.lp")
        return master

    def run_column_generation(self, max_iterations=50):
        """Run column generation with graph-based pricing"""
        for _ in range(max_iterations):
            master = self._create_master_problem()
            master.optimize()
            
            if master.status != gp.GRB.OPTIMAL:
                break

            # Get dual values
            duals = {c.ConstrName: c.Pi for c in master.getConstrs()}
            
            # Solve pricing problems
            added = 0
            for antenna in self.antennas:
                dual_values = [
                    duals.get(f"job_{node.identifier}", 0)
                    for node in self.nos[antenna][1:-1]  # Skip virtual nodes
                ]
                
                self.buckets[antenna].set_duals(dual_values)
                labels = self.buckets[antenna].solve()
                
                for label in labels:
                    column = [
                        self.nos[antenna][i].identifier 
                        for i in label.nodes_covered 
                        if self.nos[antenna][i].identifier not in {f"vs_{antenna}", f"ve_{antenna}"}
                    ]
                    if column and column not in self.colunas[antenna]:
                        self.colunas[antenna].append(column)
                        added += 1
            
            if added == 0:
                break

    def get_solution(self):
        """Retrieve final antenna schedules"""
        return {antenna: cols for antenna, cols in self.colunas.items()}
    
    def print_instance(self):
        """Print the instance with antenna jobs and columns"""
        print("\n=== Antenna Jobs ===")
        for antenna, jobs in self.antenna_jobs.items():
            print(f"\nAntenna {antenna}:")
            for job in jobs:
                print(f"  Track ID: {job['track_id']}, Duration: {job['duration']}min, Windows: {job['resource_windows']}")
        
        print("\n=== Columns ===")
        for antenna, cols in self.colunas.items():
            print(f"\nAntenna {antenna}:")
            for col in cols:
                print(f"  Column: {col}")

    def print_nodes(self):
        """Print the nodes of the bucket graphs"""
        for antenna, nodes in self.nos.items():
            print(f"\nNodes for antenna {antenna}:")
            for node in nodes:
                print(f"  Node ID: {node.id}, Identifier: {node.identifier}, LB: {node.lb}, UB: {node.ub}")
        print("\n=== Bucket Graphs ===")
        for antenna, bg in self.buckets.items():
            print(f"\nBucket graph for antenna {antenna}:")
            print(f"  Size: {bg.size}, Max time: {bg.max_time}")
            print(f"  Depot: {bg.depot}, End depot: {bg.end_depot}")
            print(f"  Resource type: {bg.resource_type}")
            print(f"  Bucket fixing: {bg.bucket_fixing}")



In [6]:
instance = Instance()
instance.load_data("build/toy_problem.json")
graph = Graph(instance)

In [9]:
scheduler = ColumnGenerationScheduler(graph)
scheduler.print_instance()
scheduler.run_column_generation()
# solution = scheduler.get_solution()

Node ef05fbcd-5daf-4adf-9419-fedd7b225576 created for antenna DSS-36 with windows [3181.0, 3206.0, 3231.0] to [3243.0, 3385.0, 3340.0]
Node c3ced005-6acf-4d99-9a2e-3b9f5927a55d created for antenna DSS-36 with windows [9330.0, 9354.0, 9378.0] to [9431.0, 9481.0, 9607.0]
Node 5dd1534c-ebeb-4efe-9599-e79b04029e73 created for antenna DSS-36 with windows [1423.0, 1451.0, 1479.0] to [1497.0, 1630.0, 1605.0]
Node a77c28a6-d6e0-498a-b9a3-fba0a46c5f9b created for antenna DSS-36 with windows [2545.0, 2565.0, 2585.0] to [2724.0, 2665.0, 2807.0]
Node 43e6f883-eb75-403d-bb19-13a4ac9e5d73 created for antenna DSS-36 with windows [4180.0, 4201.0, 4222.0] to [4419.0, 4277.0, 4363.0]
Node 3b047a8b-d4b6-42b2-9283-e03fd245d2a8 created for antenna DSS-36 with windows [3100.0, 3127.0, 3154.0] to [3267.0, 3217.0, 3327.0]
Node b5e6af44-570c-4b92-a125-5d376330c248 created for antenna DSS-36 with windows [8370.0, 8399.0, 8428.0] to [8483.0, 8593.0, 8541.0]
Node f22b543e-1df3-4a6a-b2eb-946ba4b70432 created for a

: 